**1. Importing Necessary libraries and  Made a class of Kalman Filter**

In [98]:
import csv
import math
import pandas as pd
from numpy import genfromtxt
import numpy as np
import matplotlib.pyplot as plt
import math

class KalmanFilter:

    cov = float('nan')
    x = float('nan')

    def __init__(self, R, Q):
        """
        Constructor
        :param R: Process Noise
        :param Q: Measurement Noise
        """
        self.A = 1
        self.B = 0
        self.C = 1

        self.R = R
        self.Q = Q

    def filter(self, measurement):
        """
        Filters a measurement
        :param measurement: The measurement value to be filtered
        :return: The filtered value
        """
        u = 0
        if math.isnan(self.x):
            self.x = (1 / self.C) * measurement
            self.cov = (1 / self.C) * self.Q * (1 / self.C)
        else:
            predX = (self.A * self.x) + (self.B * u)
            predCov = ((self.A * self.cov) * self.A) + self.R

            # Kalman Gain
            K = predCov * self.C * (1 / ((self.C * predCov * self.C) + self.Q));

            # Correction
            self.x = predX + K * (measurement - (self.C * predX));
            self.cov = predCov - (K * self.C * predCov);

        return self.x

    def last_measurement(self):
        """
        Returns the last measurement fed into the filter
        :return: The last measurement fed into the filter
        """
        return self.x

    def set_measurement_noise(self, noise):
        """
        Sets measurement noise
        :param noise: The new measurement noise
        """
        self.Q = noise

    def set_process_noise(self, noise):
        """
        Sets process noise
        :param noise: The new process noise
        """
        self.R = noise


**2. Combining datasets and preprocessing**

The os.listdir() function is used to list all the files in the data_folder. The if file_name.endswith('.csv') condition ensures that only CSV files are processed. The os.path.join() function is used to construct the full file path to each CSV file, and the pd.read_csv() function is used to read the CSV file into a DataFrame

In [99]:
import os

actual_dist=1 #varies with 1m to 25 m

# Set the path to the folder containing the dataset
data_folder = 'C:/Users/LILA/RTT-RSS with Kalman/raw_dataset'

# Loop through each CSV file in the folder and read it into a DataFrame
for file_name in os.listdir(data_folder):
    if file_name.endswith(str(actual_dist)+'m.csv'):
        file_path = os.path.join(data_folder, file_name)
        df = pd.read_csv(file_path)
df.drop(['#<Time(ms)>','<Ch-MHz>','<Burst#>','<AP-SSID>','<RTT AP?>','<Successes#>'],axis=1,inplace=True)
df.rename(columns={'<True Range(m)>': 'True_Range', '<Est. Range(m)>': 'Est_Range','<RSSI(dBm)>': 'RSSI','<Std dev(m)>': 'stdev','<Successes#>': 'successes'}, inplace=True)
Est=df.Est_Range
R_RTT=Est.var()
data1_RTT=Est.to_list()
df

,True_Range,Est_Range,stdev,RSSI
0,21,21.057,0.444,-76
1,21,21.233,0.551,-74
2,21,21.291,0.875,-74
3,21,20.940,0.673,-72
4,21,21.291,0.460,-73
...,...,...,...,...
294,21,21.087,0.444,-78
295,21,21.028,0.576,-77
296,21,22.053,0.061,-79
297,21,20.969,0.576,-79


In [100]:
test = KalmanFilter(0.09,R_RTT)
testData =data1_RTT
data=[] # for storing all data
for x in testData:
    #data2= test.filter(x)
    #print ("Filtered Data:",data2)
    data.append(test.filter(x)) 
output=list(map(float,data))    #making it in the list
output_RTT=np.array(output)
#print(output)
#print(data)
output_RTT.shape

(299,)

In [101]:
RSSI=df.RSSI
R_RSSI=RSSI.var()
data1_RSS=RSSI.to_list()
#data2=np.array(data1) #converting list to array
#data2

In [102]:
test = KalmanFilter(0.09,R_RSSI)
testData =data1_RSS
data=[] # for storing all data
for x in testData:
    data2= test.filter(x)
    #print ("Filtered Data:",data2)
    data.append(test.filter(x)) 
output=list(map(float,data))    #making it in the list
output_RSS=np.array(output)
#print(output_RSS)
output_RSS.shape

(299,)

In [103]:
# Define the column names for the DataFrame
columns = ['RTT', 'RSS']

# Create the DataFrame from the two separate lists
df_final = pd.DataFrame({'RTT': output_RTT, 'RSS': output_RSS})

# Specify the file name and path where you want to save the file
file_path = 'Kalman_'+str(actual_dist)+'m.csv'

# Save the DataFrame to a CSV file using the to_csv() method
df_final.to_csv(file_path, index=False)

print(f'File saved to {file_path}!')


File saved to Kalman_1m.csv!
